In [1]:
import geopandas as gpd
import pandas as pd
from calculo_predial import *
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *

## Valor Catastral
El valor catastral es la suma de los valores catastrales (terreno y construcion)


Vcat = VCT + VCC

Vcat = Valor catastral <br>
VCT = Valor catastral del terreno <br>
VCC = Valor catastral de la construcción <br>

## Valor Catastral del terreno

factor irregular = Revisar los poligonos que se intersectan | asumir 

identificar los multiples y separarlos

luego hacer 


In [ ]:
factor_frente(frente) * factor_fondo(fondo,fondo_base) * factor_irregularidad(superficie, area_inscrita) * factor_area(superficie, )